In [5]:
import pandas as pd
from langdetect import detect

In [6]:
notes = pd.read_parquet('rated_notes_compact.parquet', engine='auto')

In [7]:
# clumn names
notes.columns

Index(['noteId', 'finalRatingStatus', 'firstTag', 'secondTag',
       'classification', 'createdAtMillis', 'numRatings', 'noteTopic',
       'topicNoteConfident', 'noteAuthorParticipantId', 'tweetId',
       'misleadingOther', 'misleadingFactualError',
       'misleadingManipulatedMedia', 'misleadingOutdatedInformation',
       'misleadingMissingImportantContext', 'misleadingUnverifiedClaimAsFact',
       'misleadingSatire', 'notMisleadingOther',
       'notMisleadingFactuallyCorrect',
       'notMisleadingOutdatedButNotWhenWritten', 'notMisleadingClearlySatire',
       'notMisleadingPersonalOpinion', 'trustworthySources', 'summary',
       'isMediaNote'],
      dtype='object')

In [8]:
# number of rated notes
len(notes)

175861

check assigned topics

In [9]:
notes['noteTopic'].value_counts()

noteTopic
GazaConflict       9540
UkraineConflict    5970
MessiRonaldo        514
Name: count, dtype: int64

In [10]:
notes['noteTopic'].unique()

array([None, 'GazaConflict', 'UkraineConflict', 'MessiRonaldo'],
      dtype=object)

check how many does not have an assigned topic

In [11]:
len(notes[notes['noteTopic'].values == None])

159837

Most of them does not have an assigned topic! Let's see 10 random note without a topic!


In [12]:
notes[notes['noteTopic'].values == None]['summary'].sample(n=10)

49015                                                  None
124020    トランプ氏は自身が立ち上げたSNS「True Social」上で撃たれたのは右耳の上部である...
101937    This is a graphic from 2006.   https://www.reu...
28545     It’s not directly by Honda, there is a car del...
166917    This post contains an unsolicited advertisemen...
163890    Lewis Hamilton never said that. The author of ...
41035                                                  None
25204     The CDC said that even people who have immunit...
15524                                                  None
7944      Trata-se de uma encenação. O criador do vídeo ...
Name: summary, dtype: object

It seems like most of them are not in English! Plus, some of them are None! Let's remove the None s and detect the language the notes were written in!

In [45]:
notes = notes[notes['summary'].notna()]

In [46]:
len(notes)

159226

In [51]:
# using detect function from langdetect to create a new column for languages
langs = []
for note in notes['summary']:
    try:
        langs.append(detect(note))
    # except LangDetectException:
    #     print(f"LangDetectException for note: {note}")
    #     langs.append(None)
    except Exception as error:
    # handle the exception
        print("An exception occurred:", error) 
        print(note)
        langs.append(None)

An exception occurred: No features in text.
🖕🖕🖕🖕🖕🖕
An exception occurred: No features in text.
http://www.shadowstats.com/alternate_data/inflation-charts
An exception occurred: No features in text.
🫡
An exception occurred: No features in text.
🔥 🐶☕ 🔥
An exception occurred: No features in text.
.
An exception occurred: No features in text.
https://www.mhlw.go.jp/stf/seisakunitsuite/bunya/vaccine_pfizer_bivalent.html#002
An exception occurred: No features in text.
https://sites.imsa.edu/acronym/2021/03/04/elon-musk-is-not-an-entrepreneur-hes-a-rich-deceitful-hack/  https://www.cnbc.com/2018/06/01/elon-musks-mom-worked-5-jobs-to-raise-3-kids-after-her-divorce.html  https://www.cnbc.com/2018/06/12/telsas-elon-musk-tweets-he-arrived-in-north-america-at-17-with-2000.html  
An exception occurred: No features in text.
https://twitter.com/horacioneiva/status/1658638214385614852?s=20
An exception occurred: No features in text.
https://www.washingtonpost.com/politics/2021/03/19/how-spot-fake-vide

In [54]:
notes['language'] = langs

There are some notes without a language. They may contain external links, emojis, numbers, punctuation marks or a mixture of them.